### Select patients for CFR model: Split patients in train, val and test sets ###

In [1]:
import os
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [2]:
cfr_data_root = os.path.normpath('/mnt/obi0/andreas/data/cfr')
match_view_filename = '210_getStressTest_match365_files_BWH_200131.parquet'
files_cfr = pd.read_parquet(os.path.join(cfr_data_root, 'metadata_200131', match_view_filename))

print('Total number of patients {}'.format(len(files_cfr.mrn.unique())))
print('Total number of studies  {}'.format(len(files_cfr.study.unique())))
print('Total number of files    {}'.format(len(files_cfr.filename.unique())))

files_cfr.head()

Total number of patients 1792
Total number of studies  2921
Total number of files    131290


,mrn,study,echo_study_date,reportID,cfr_days_after_echo,subjectid,report_number,cfr_study_date,cfr_report_date,cfr,filename,dir,datetime,fileid,institution,model,manufacturer,index,frame_time,number_of_frames,heart_rate,deltaX,deltaY,view_a2c,view_a3c,view_a4c,view_plax,view_psax,view_other,sum_views,max_view
0,9241,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e,2010-04-02,100820,-64,121,EVS0228610,2010-01-28,2010-01-26,1.133683,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,/mnt/obi0/phi/echo/npyFiles/BWH/4b7b/4b7b463ae...,2010-04-02 07:53:16,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,BWH,Vivid7,GE Vingmed Ultrasound,0.0,40.089256,56.0,85.0,0.027944,0.027944,9.762369e-01,2.330583e-04,3.340996e-03,1.106907e-04,7.710441e-04,1.930706e-02,1.0,view_a2c
1,9241,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e,2010-04-02,100820,-64,121,EVS0228610,2010-01-28,2010-01-26,1.133683,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,/mnt/obi0/phi/echo/npyFiles/BWH/4b7b/4b7b463ae...,2010-04-02 07:53:16,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,BWH,Vivid7,GE Vingmed Ultrasound,0.0,40.064101,53.0,88.0,0.017671,0.017671,1.080687e-06,2.221534e-07,1.344890e-06,9.993535e-01,6.972564e-06,6.367801e-04,1.0,view_plax
2,9241,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e,2010-04-02,100820,-64,121,EVS0228610,2010-01-28,2010-01-26,1.133683,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,/mnt/obi0/phi/echo/npyFiles/BWH/4b7b/4b7b463ae...,2010-04-02 07:53:16,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,BWH,Vivid7,GE Vingmed Ultrasound,0.0,48.333332,46.0,107.0,0.076726,0.076726,2.892548e-11,2.382341e-09,3.241086e-09,1.000000e+00,2.436432e-10,4.737633e-10,1.0,view_plax
3,9241,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e,2010-04-02,100820,-64,121,EVS0228610,2010-01-28,2010-01-26,1.133683,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,/mnt/obi0/phi/echo/npyFiles/BWH/4b7b/4b7b463ae...,2010-04-02 07:53:16,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,BWH,Vivid7,GE Vingmed Ultrasound,0.0,40.450451,38.0,84.0,0.043312,0.043312,3.644072e-10,1.350347e-10,1.540048e-07,1.887208e-10,1.837416e-07,9.999996e-01,1.0,view_other
4,9241,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e,2010-04-02,100820,-64,121,EVS0228610,2010-01-28,2010-01-26,1.133683,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,/mnt/obi0/phi/echo/npyFiles/BWH/4b7b/4b7b463ae...,2010-04-02 07:53:16,4b7b463ae45ad0e0_4903a585842a266f561aee3f563e_...,BWH,Vivid7,GE Vingmed Ultrasound,0.0,40.119048,57.0,85.0,0.020879,0.020879,1.801199e-13,9.366453e-13,6.398218e-12,3.151624e-12,6.481914e-12,1.000000e+00,1.0,view_other


### Split the patients in train, validate and test sets ###
Although each view might have a little different patient pupulation distribution, because not all views are in each study. However, we want the same MRNS in each data set and for all views so that we can directly compare the performance of the algorithm for the same patients. We can expand the data frame above to add the splits.

In [3]:
def patientsplit(patient_list):

    train_test_split = 0.86
    train_eval_split = 0.90

    # Split train/test sets
    patient_list_train = np.random.choice(patient_list,
                                          size = int(np.floor(train_test_split*len(patient_list))),
                                          replace = False)
    patient_list_test = list(set(patient_list).difference(patient_list_train))
    train_test_intersection = set(patient_list_train).intersection(set(patient_list_test)) # This should be empty
    print('Intersection of patient_list_train and patient_list_test:', train_test_intersection)

    # Further separate some patients for evaluation
    patient_list_eval = np.random.choice(patient_list_train,
                                         size = int(np.ceil((1-train_eval_split)*len(patient_list_train))),
                                         replace = False)

    patient_list_train = set(patient_list_train).difference(patient_list_eval)
    train_eval_intersection = set(patient_list_train).intersection(set(patient_list_eval))
    print('Intersection of patient_list_train and patient_list_eval:', train_eval_intersection)

    # Show the numbers
    print('total patients:', len(patient_list))
    print('patients in set:', np.sum([len(patient_list_train),
                                     len(patient_list_eval),
                                     len(patient_list_test)]))
    print('patients in train:', len(patient_list_train))
    print('patients in eval:', len(patient_list_eval))
    print('patients in test:', len(patient_list_test))

    return patient_list_train, patient_list_eval, patient_list_test

In [4]:
# Get a patient list
patient_list = list(files_cfr.sample(frac=1).mrn.unique())
patient_list_train, patient_list_eval, patient_list_test = patientsplit(patient_list)

patient_split = {'train': patient_list_train,
                 'eval': patient_list_eval,
                 'test': patient_list_test}

print('Patient IDs in train:', len(patient_split['train']))
print('Patient IDs in eval:', len(patient_split['eval']))
print('Patient IDs in test:', len(patient_split['test']))

print()

print('contamination train-test:', set(patient_split['train']).intersection(set(patient_split['test'])))
print('contamination train-eval:', set(patient_split['train']).intersection(set(patient_split['eval'])))
print('contamination eval-test:', set(patient_split['eval']).intersection(set(patient_split['test'])))

Intersection of patient_list_train and patient_list_test: set()
Intersection of patient_list_train and patient_list_eval: set()
total patients: 1792
patients in set: 1792
patients in train: 1386
patients in eval: 155
patients in test: 251
Patient IDs in train: 1386
Patient IDs in eval: 155
Patient IDs in test: 251

contamination train-test: set()
contamination train-eval: set()
contamination eval-test: set()


In [5]:
# Add dset column to files_cfr
split_list = []
for dset in patient_split.keys():
    dset_mrn_list = list(patient_split[dset])
    split_list.append(pd.DataFrame({'mrn': dset_mrn_list,
                                    'dset': [dset]*len(dset_mrn_list)}))
split_df = pd.concat(split_list, ignore_index = True)

files_cfr_dset = files_cfr.merge(right = split_df, on = 'mrn', how = 'left').\
                    sample(frac = 1).\
                    reset_index(drop = True)

In [6]:
files_cfr_dset.head(50)

,mrn,study,echo_study_date,reportID,cfr_days_after_echo,subjectid,report_number,cfr_study_date,cfr_report_date,cfr,filename,dir,datetime,fileid,institution,model,manufacturer,index,frame_time,number_of_frames,heart_rate,deltaX,deltaY,view_a2c,view_a3c,view_a4c,view_plax,view_psax,view_other,sum_views,max_view,dset
0,22628267,490173dde6391a15_4903a44b32edbb664d548b67cd2a,2008-01-31,73245,1,3638,0803132W,2008-02-01,2008-01-31,1.622875,490173dde6391a15_4903a44b32edbb664d548b67cd2a_...,/mnt/obi0/phi/echo/npyFiles/BWH/4901/490173dde...,2008-01-31 13:22:05,490173dde6391a15_4903a44b32edbb664d548b67cd2a_...,BWH,Vivid i,GEMS Ultrasound,0.0,56.350378,36.0,57.0,0.046036,0.046036,3.437009e-10,5.613545e-09,5.215388e-12,6.688370e-09,2.574061e-08,1.000000e+00,1.0,view_other,train
1,28940807,490b13fe1c0cfe48_4903a581a72a3bb34bc3edd1a7d7,2014-01-29,129567,14,5799,EVS0459128,2014-02-12,2014-01-29,1.865636,490b13fe1c0cfe48_4903a581a72a3bb34bc3edd1a7d7_...,/mnt/obi0/phi/echo/npyFiles/BWH/490b/490b13fe1...,2014-01-29 10:23:17,490b13fe1c0cfe48_4903a581a72a3bb34bc3edd1a7d7_...,BWH,iE33,Philips Medical Systems,0.0,57.353000,35.0,62.0,0.041844,0.041844,5.679472e-09,1.795165e-09,8.519532e-08,1.852653e-09,1.076658e-08,9.999999e-01,1.0,view_other,eval
2,24827024,4907b1bbe2476ccb_4903a585842a24fc91e68a82c756,2010-04-26,103187,17,261,EVS0243604,2010-05-13,2010-04-26,1.821608,4907b1bbe2476ccb_4903a585842a24fc91e68a82c756_...,/mnt/obi0/phi/echo/npyFiles/BWH/4907/4907b1bbe...,2010-04-26 14:56:12,4907b1bbe2476ccb_4903a585842a24fc91e68a82c756_...,BWH,Vivid E9,GE Vingmed Ultrasound,0.0,40.833333,45.0,106.0,0.038417,0.038417,3.957838e-04,6.696503e-06,3.248699e-04,4.975402e-02,2.004606e-04,9.493180e-01,1.0,view_other,train
3,6199327,4b7d757323c4a9b7_4903a44ab09601d997624b4d023d,2009-11-17,92847,-287,466,EVS0216921,2009-02-03,2009-11-17,1.647627,4b7d757323c4a9b7_4903a44ab09601d997624b4d023d_...,/mnt/obi0/phi/echo/npyFiles/BWH/4b7d/4b7d75732...,2009-11-17 15:17:42,4b7d757323c4a9b7_4903a44ab09601d997624b4d023d_...,BWH,Vivid7,GE Vingmed Ultrasound,0.0,40.333332,56.0,85.0,0.048593,0.048593,6.729118e-11,5.173809e-10,1.801301e-09,9.316732e-11,4.427915e-10,1.000000e+00,1.0,view_other,train
4,26648600,490530d879da4faa_4903a58793bfcc33959eeb94be4a,2012-08-27,115228,-199,4813,STR11869367,2012-02-10,2012-08-27,2.001497,490530d879da4faa_4903a58793bfcc33959eeb94be4a_...,/mnt/obi0/phi/echo/npyFiles/BWH/4905/490530d87...,2012-08-27 09:25:41,490530d879da4faa_4903a58793bfcc33959eeb94be4a_...,BWH,ACUSON SC2000,SIEMENS,0.0,11.891000,22.0,61.0,-0.026667,0.026667,1.772770e-06,2.005140e-03,1.807704e-05,7.184519e-02,2.000867e-04,9.259297e-01,1.0,view_other,train
5,22144026,49017450fc75abc7_4903a58051d59d4788a62afe2868,2015-11-06,145874,164,6742,E1275779,2016-04-18,2015-11-06,2.539548,49017450fc75abc7_4903a58051d59d4788a62afe2868_...,/mnt/obi0/phi/echo/npyFiles/BWH/4901/49017450f...,2015-11-06 14:13:35,49017450fc75abc7_4903a58051d59d4788a62afe2868_...,BWH,iE33,Philips Medical Systems,0.0,19.838000,69.0,84.0,0.033177,0.033177,7.916762e-11,4.809451e-11,2.316182e-11,7.478193e-12,1.000000e+00,1.044415e-09,1.0,view_psax,train
6,20248746,4903a67fc18a3ff5_4903a581a68027e91001ace61358,2014-11-24,136035,14,6027,EVS0555147,2014-12-08,2015-02-23,2.012987,4903a67fc18a3ff5_4903a581a68027e91001ace61358_...,/mnt/obi0/phi/echo/npyFiles/BWH/4903/4903a67fc...,2014-11-24 13:37:46,4903a67fc18a3ff5_4903a581a68027e91001ace61358_...,BWH,ACUSON SC2000,SIEMENS,0.0,66.523500,4.0,103.0,-0.026667,0.026667,8.092341e-02,1.158805e-04,2.553214e-04,3.566241e-02,3.721472e-01,5.108957e-01,1.0,view_other,train
7,21322607,490289f920e53080_4903a444c23a2b4adc7cabfac881,2007-10-10,71765,54,3569,EVS0082945,2007-12-03,2007-10-10,2.080271,490289f920e53080_4903a444c23a2b4adc7cabfac881_...,/mnt/obi0/phi/echo/npyFiles/BWH/4902/490289f92...,2007-10-10 09:29:06,490289f920e53080_4903a444c23a2b4adc7cabfac881_...,BWH,SEQUOIA,ACUSON,0.0,33.333000,9.0,112.0,1.000000,1.000000,5.108089e-03,9.948118e-01,8.659935e-08,1.441704e-06,6.549722e-07,7

In [7]:
dset_filename = '210_getStressTest_files_dset_BWH_200131.parquet'
files_cfr_dset.to_parquet(os.path.join(cfr_data_root, 'metadata_200131', dset_filename))